In [163]:
from dotenv import load_dotenv
import json
import numpy as np
import pandas as pd
import os

In [2]:
load_dotenv()
NREL_API_KEY = os.getenv("NREL_API_KEY")
NREL_API_EMAIL = os.getenv("NREL_API_EMAIL")

In [3]:
import sys

sys.path.append("/Users/andrebach/code/solmap")
%load_ext autoreload
%autoreload 2

In [4]:
import data_generation

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [156]:
new_summary = data_generation.geometry.load_congresional_district_points(
    file_to_save="congressional_geometry_v4.csv"
)

i = 0
i = 100
i = 200
i = 300
i = 400
Wrote to congressional_geometry_v4.csv


In [160]:
mexico_df = data_generation.geometry.load_mexico_district_points(
    file_to_save="mexico_geometry_v2.csv",
)

Not bothering with country_mexico-state_Oaxaca-district_Matías Romero Avendaño
Not bothering with country_mexico-state_Sonora-district_Guaymas
Wrote to mexico_geometry_v2.csv


In [164]:
with open("mexico.json", "r") as f:
    mexico_json = json.load(f)

In [236]:
def arbitrary_coords_lists_to_array(coords, verbose=False):
    """
    GeoJSON polygons from the internet are in inconsistently formatted
    lists of lists of lists (of lists). We want a simple array of shape (N, 2).
    This flattens such an array.
    """
    if verbose:
        print(f"Starting {len(coords)=}")
    flat = []
    for item in coords:
        if verbose:
            print(f"    Processing {len(item)=}")
        subitem_lengths_all_2 = all([len(i) == 2 for i in item])
        subsubitems_all_floats = all(
            [all([isinstance(j, (float, int)) for j in i]) for i in item]
        )
        if verbose:
            print(f"    {subitem_lengths_all_2=}, {subsubitems_all_floats=}")
        if subitem_lengths_all_2 and subsubitems_all_floats:
            if verbose:
                print(f"        Appending item {len(item)=}")
            for pair in item:
                flat.append(pair)
        else:
            if verbose:
                print("        Recursive call")
            flat.extend(arbitrary_coords_lists_to_array(item, verbose=verbose))
    return flat

In [240]:
for feature in mexico_json["features"]:
    properties = feature["properties"]
    if not properties["NOMGEO"] == "Mexicali":
       continue
    coordinates = feature["geometry"]["coordinates"]
    nice_array = np.array(
        arbitrary_coords_lists_to_array(coordinates, verbose=True)
    )
    assert nice_array.ndim == 2
    assert nice_array.shape[1] == 2

Starting len(coords)=2
    Processing len(item)=1
    subitem_lengths_all_2=False, subsubitems_all_floats=False
        Recursive call
Starting len(coords)=1
    Processing len(item)=295
    subitem_lengths_all_2=True, subsubitems_all_floats=True
        Appending item len(item)=295
    Processing len(item)=1
    subitem_lengths_all_2=False, subsubitems_all_floats=False
        Recursive call
Starting len(coords)=1
    Processing len(item)=5579
    subitem_lengths_all_2=True, subsubitems_all_floats=True
        Appending item len(item)=5579


In [238]:
nice_array.shape

(17838, 2)

In [239]:
nice_array[:10]

array([[-110.95087805,   27.8673884 ],
       [-110.95108619,   27.86737384],
       [-110.95124749,   27.86750879],
       [-110.95124224,   27.86776991],
       [-110.95124014,   27.8679799 ],
       [-110.95120758,   27.86804466],
       [-110.9510761 ,   27.86809394],
       [-110.95098054,   27.8681742 ],
       [-110.95094546,   27.8682511 ],
       [-110.95088812,   27.86829925]])

In [235]:
nice_array.std(axis=0)

array([0.24890233, 0.1436327 ])

In [230]:
nice_array.std(axis=0)

array([0.24890233, 0.1436327 ])

In [220]:
for feature in mexico_json["features"][:3]:
    properties = feature["properties"]
    coordinates = feature["geometry"]["coordinates"]
    nice_array = np.array(
        arbitrary_coords_lists_to_array(coordinates, verbose=True)
    )
    assert nice_array.ndim == 2
    assert nice_array.shape[1] == 2
    print()

Starting len(coords)=1
    Processing len(item)=817
    subitem_lengths_all_2=True, subsubitems_all_floats=True
        Appending item len(item)=817

Starting len(coords)=1
    Processing len(item)=851
    subitem_lengths_all_2=True, subsubitems_all_floats=True
        Appending item len(item)=851

Starting len(coords)=1
    Processing len(item)=711
    subitem_lengths_all_2=True, subsubitems_all_floats=True
        Appending item len(item)=711



In [221]:
nice_array.shape

(711, 2)